# Proposal

### Answer Questions Here

blah blah blah

### Glass Dataset

**Type of Glass** \
1 : Building Windows (float-processed) \
2 : Building Windows (non-float-processed) \
3 : Vehicle Windows (float-processed) \
4 : Vehicle Windows (non-float-processed) *none in this dataset* \
5 : Containers \
6 : Tableware \
7 : Headlamps 

Source: https://www.kaggle.com/datasets/uciml/glass?select=glass.csv

In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

In [ ]:
glass <- read_csv("glass.csv")
glass